<a href="https://colab.research.google.com/github/backlashblitz/Bangla-Book-Recommendation-Dataset/blob/main/colabnotebooks/Two_Tower_Text_Embedding_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, shutil, subprocess

subprocess.run(["pip", "install", "-q", "huggingface_hub"], check=True)

from huggingface_hub import snapshot_download

REPO_ID = "DevnilMaster1/Bangla-Book-Recommendation-Dataset"
DATA_FOLDER = "RokomariBG_Dataset"

# Download the full dataset repo snapshot
print("Downloading dataset from HuggingFace Hub...")
local_repo = snapshot_download(repo_id=REPO_ID, repo_type="dataset")
print(f"Repo downloaded to: {local_repo}")

FILES_NEEDED = [
    "author.json",
    "book.json",
    "category.json",
    "publisher.json",
    "review.json",
    "book_to_author.json",
    "book_to_category.json",
    "book_to_publisher.json",
    "book_to_review.json",
    "user_to_review.json",
]

os.makedirs(DATA_FOLDER, exist_ok=True)

# Walk the downloaded repo and find each needed file
found = {}
for root, dirs, files in os.walk(local_repo):
    for fname in files:
        if fname in FILES_NEEDED and fname not in found:
            found[fname] = os.path.join(root, fname)

for filename in FILES_NEEDED:
    dest = os.path.join(DATA_FOLDER, filename)
    if filename in found:
        shutil.copy(found[filename], dest)
        print(f"✅ Saved: {filename}")
    else:
        print(f"⚠️ Not found in repo: {filename}")

base_dir = DATA_FOLDER
print("\n🎉 Environment Ready!")

In [ ]:
# two_tower_rokomari.py
# pip install torch sentence-transformers pandas numpy tqdm scikit-learn

import os, json, math, random
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer


# ----------------------------
# Config
# ----------------------------
@dataclass
class Paths:
    author_json: str
    book_json: str
    category_json: str
    publisher_json: str
    review_json: str
    book_to_author_json: str
    book_to_category_json: str
    book_to_publisher_json: str
    book_to_review_json: str
    user_to_review_json: str

@dataclass
class TrainConfig:
    seed: int = 42
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    # embedding sizes
    id_emb_dim: int = 128
    text_emb_dim: int = 256  # multilingual-e5-base outputs 768; we can project down
    out_dim: int = 256

    # tower MLP hyperparams
    item_mlp_layers: int = 2
    user_mlp_layers: int = 2
    item_hidden_dim: int = 256
    user_hidden_dim: int = 256
    dropout: float = 0.1
    use_layernorm: bool = True


    # training
    batch_size: int = 256
    epochs: int = 20
    lr: float = 2e-3
    weight_decay: float = 1e-5
    max_history: int = 50  # for user history pooling

    # eval
    k_list: Tuple[int, ...] = (10, 20)
    num_neg_eval: int = 200  # sampled negatives per user for fast eval

    # text model
    text_model_name: str = "intfloat/multilingual-e5-large-instruct"
    text_cache_dir: str = "./text_cache"

    # early stopping / selection
    patience: int = 2  # stop if no val NDCG@10 improvement for this many epochs

    # evaluation
    eval_ks: Tuple[int, ...] = (5, 10, 50)
    mrr_k: int = 10
    ndcg_ks: Tuple[int, ...] = (10, 50)

    # ---- Signal toggles (leave-one-out ablations) ----
    use_interaction: bool = True     # user-item interaction / history pooling signal
    use_side: bool = True            # side features = text + numeric
    use_relations: bool = True       # relational knowledge = author/cat/pub
    ablation_name: str = "full"




def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# ----------------------------
# Utilities: load JSON arrays
# ----------------------------
def load_json_array(path: str) -> List[dict]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def safe_str(x) -> str:
    if x is None:
        return ""
    return str(x)

def normalize_numeric(df: pd.DataFrame, cols: List[str]) -> Tuple[pd.DataFrame, Dict[str, Tuple[float,float]]]:
    stats = {}
    for c in cols:
        v = pd.to_numeric(df[c], errors="coerce")
        mean = float(v.mean(skipna=True)) if v.notna().any() else 0.0
        std = float(v.std(skipna=True)) if v.notna().any() else 1.0
        if std == 0.0: std = 1.0
        df[c] = (v.fillna(mean) - mean) / std
        stats[c] = (mean, std)
    return df, stats


# ----------------------------
# Build interaction table: (user_id, book_id) via review_id join
# ----------------------------
def build_interactions(paths: Paths) -> pd.DataFrame:
    reviews = pd.DataFrame(load_json_array(paths.review_json))
    reviews["review_id"] = pd.to_numeric(reviews["review_id"], errors="coerce").astype("Int64")

    u2r = pd.DataFrame(load_json_array(paths.user_to_review_json))
    b2r = pd.DataFrame(load_json_array(paths.book_to_review_json))

    u2r["review_id"] = pd.to_numeric(u2r["review_id"], errors="coerce").astype("Int64")
    b2r["review_id"] = pd.to_numeric(b2r["review_id"], errors="coerce").astype("Int64")
    b2r["book_id"] = b2r["book_id"].astype(str)

    tmp = u2r.merge(reviews, on="review_id", how="left")
    inter = tmp.merge(b2r[["book_id","review_id"]], on="review_id", how="inner")

    inter = inter.dropna(subset=["user_id", "book_id"])
    inter["user_id"] = inter["user_id"].astype(str)

    inter["verified_purchase"] = inter.get("verified_purchase", False).fillna(False).astype(bool)
    inter["user_rating"] = pd.to_numeric(inter.get("user_rating", np.nan), errors="coerce")

    w = np.ones(len(inter), dtype=np.float32)
    w += inter["verified_purchase"].astype(np.float32) * 0.5
    r = inter["user_rating"].fillna(0).to_numpy()
    r = np.clip(r / 5.0, 0, 1).astype(np.float32)
    w += 0.2 * r
    inter["weight"] = w

    inter = inter.sort_values("weight", ascending=False).drop_duplicates(["user_id","book_id"], keep="first")
    return inter[["user_id","book_id","weight","review_id"]].reset_index(drop=True)


# ----------------------------
# Build item (book) features
# ----------------------------
def build_book_tables(paths: Paths) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    books = pd.DataFrame(load_json_array(paths.book_json))
    authors = pd.DataFrame(load_json_array(paths.author_json))
    cats = pd.DataFrame(load_json_array(paths.category_json))
    pubs = pd.DataFrame(load_json_array(paths.publisher_json))
    reviews = pd.DataFrame(load_json_array(paths.review_json))

    books["book_id"] = books["book_id"].astype(str)
    books = books.drop_duplicates(subset=["book_id"], keep="first").reset_index(drop=True)
    if "author_id" in books.columns:
        books["author_id"] = books["author_id"].astype(str)
    if "publisher_id" in books.columns:
        books["publisher_id"] = books["publisher_id"].astype(str)
    if "category_id" in books.columns:
        books["category_id"] = books["category_id"].astype(str)

    if "author_id" in authors.columns:
        authors["author_id"] = authors["author_id"].astype(str)
        authors = authors.drop_duplicates(subset=["author_id"], keep="first").reset_index(drop=True)
    if "publisher_id" in authors.columns:
        authors["publisher_id"] = authors["publisher_id"].astype(str)

    cats["category_id"] = cats["category_id"].astype(str)
    cats = cats.drop_duplicates(subset=["category_id"], keep="first").reset_index(drop=True)
    pubs["publisher_id"] = pubs["publisher_id"].astype(str)
    pubs = pubs.drop_duplicates(subset=["publisher_id"], keep="first").reset_index(drop=True)

    reviews["review_id"] = pd.to_numeric(reviews["review_id"], errors="coerce").astype("Int64")
    return books, authors, cats, pubs, reviews


def build_multi_maps(paths: Paths) -> Tuple[Dict[str, List[str]], Dict[str, List[str]], Dict[str, str]]:
    b2a = pd.DataFrame(load_json_array(paths.book_to_author_json))
    b2c = pd.DataFrame(load_json_array(paths.book_to_category_json))
    b2p = pd.DataFrame(load_json_array(paths.book_to_publisher_json))

    b2a["book_id"] = b2a["book_id"].astype(str)
    b2a["author_id"] = b2a["author_id"].astype(str)

    b2c["book_id"] = b2c["book_id"].astype(str)
    b2c["category_id"] = b2c["category_id"].astype(str)

    b2p["book_id"] = b2p["book_id"].astype(str)
    b2p["publisher_id"] = b2p["publisher_id"].astype(str)

    book_to_authors = b2a.groupby("book_id")["author_id"].apply(list).to_dict()
    book_to_cats = b2c.groupby("book_id")["category_id"].apply(list).to_dict()
    book_to_pub = b2p.groupby("book_id")["publisher_id"].first().to_dict()
    return book_to_authors, book_to_cats, book_to_pub


def make_book_text(
    book_row: pd.Series,
    authors_by_id: Dict[str, dict],
    cats_by_id: Dict[str, dict],
    pubs_by_id: Dict[str, dict],
    author_ids: List[str],
    cat_ids: List[str],
    pub_id: Optional[str]
) -> str:
    title = safe_str(book_row.get("title"))
    summary = safe_str(book_row.get("book_summary"))

    author_texts = []
    for aid in author_ids[:3]:
        a = authors_by_id.get(aid)
        if not a: continue
        author_texts.append(safe_str(a.get("author")))
        author_texts.append(safe_str(a.get("bio")))
        author_texts.append(safe_str(a.get("known_for_tokens")))

    cat_texts = []
    for cid in cat_ids[:3]:
        c = cats_by_id.get(cid)
        if not c: continue
        cat_texts.append(safe_str(c.get("category_name")))
        cat_texts.append(safe_str(c.get("category_description")))

    pub_text = ""
    if pub_id is not None:
        p = pubs_by_id.get(pub_id)
        if p:
            pub_text = " ".join([safe_str(p.get("publisher_name")), safe_str(p.get("publisher_description"))])

    blob = " ".join([title, summary, " ".join(author_texts), " ".join(cat_texts), pub_text]).strip()
    return f"passage: {blob}"


# ----------------------------
# Text embedding cache
# ----------------------------
def compute_or_load_text_embeddings(
    model_name: str,
    cache_path: str,
    texts: List[str],
    batch_size: int = 64
) -> np.ndarray:
    os.makedirs(os.path.dirname(cache_path), exist_ok=True)
    if os.path.exists(cache_path):
        return np.load(cache_path)
    model = SentenceTransformer(model_name)
    embs = model.encode(texts, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    np.save(cache_path, embs)
    return embs


# ----------------------------
# Torch Dataset
# ----------------------------
class InteractionDataset(torch.utils.data.Dataset):
    def __init__(self, inter_df: pd.DataFrame, user2idx: Dict[str,int], item2idx: Dict[str,int]):
        self.u = inter_df["user_id"].map(user2idx).to_numpy(dtype=np.int64)
        self.i = inter_df["book_id"].map(item2idx).to_numpy(dtype=np.int64)
        self.w = inter_df["weight"].to_numpy(dtype=np.float32)

    def __len__(self):
        return len(self.u)

    def __getitem__(self, idx):
        return self.u[idx], self.i[idx], self.w[idx]

def build_mlp(in_dim: int, hidden_dim: int, out_dim: int, num_layers: int, dropout: float, use_layernorm: bool) -> nn.Module:
    if num_layers < 1: raise ValueError("num_layers must be >= 1")
    layers: List[nn.Module] = []
    if num_layers == 1:
        layers.append(nn.Linear(in_dim, out_dim))
        return nn.Sequential(*layers)
    d = in_dim
    for _ in range(num_layers - 1):
        layers.append(nn.Linear(d, hidden_dim))
        if use_layernorm: layers.append(nn.LayerNorm(hidden_dim))
        layers.append(nn.ReLU())
        if dropout > 0: layers.append(nn.Dropout(dropout))
        d = hidden_dim
    layers.append(nn.Linear(d, out_dim))
    return nn.Sequential(*layers)


# ----------------------------
# Two-tower model
# ----------------------------
class TwoTower(nn.Module):
    def __init__(self, num_users, num_items, num_authors, num_cats, num_pubs, dense_dim, text_in_dim, cfg):
        super().__init__()
        self.cfg = cfg
        self.user_emb = nn.Embedding(num_users, cfg.id_emb_dim)
        self.item_emb = nn.Embedding(num_items, cfg.id_emb_dim)
        self.author_emb = nn.Embedding(max(1, num_authors), cfg.id_emb_dim)
        self.cat_emb = nn.Embedding(max(1, num_cats), cfg.id_emb_dim)
        self.pub_emb = nn.Embedding(max(1, num_pubs), cfg.id_emb_dim)
        self.global_user = nn.Parameter(torch.zeros(cfg.out_dim))
        self.text_proj = nn.Linear(text_in_dim, cfg.text_emb_dim) if cfg.use_side else None

        item_in = cfg.id_emb_dim
        if cfg.use_relations: item_in += cfg.id_emb_dim * 3
        if cfg.use_side: item_in += dense_dim + cfg.text_emb_dim

        self.item_mlp = build_mlp(item_in, cfg.item_hidden_dim, cfg.out_dim, cfg.item_mlp_layers, cfg.dropout, cfg.use_layernorm)
        user_in = cfg.id_emb_dim + cfg.out_dim
        self.user_mlp = build_mlp(user_in, cfg.user_hidden_dim, cfg.out_dim, cfg.user_mlp_layers, cfg.dropout, cfg.use_layernorm)

    def encode_item(self, item_idx, pub_idx, author_idx_list, cat_idx_list, dense, text_emb):
        it = self.item_emb(item_idx)
        pb = self.pub_emb(pub_idx)
        a_emb = self.author_emb(author_idx_list)
        c_emb = self.cat_emb(cat_idx_list)
        a_mask = (author_idx_list != 0).float().unsqueeze(-1)
        c_mask = (cat_idx_list != 0).float().unsqueeze(-1)
        a_pool = (a_emb * a_mask).sum(dim=1) / a_mask.sum(dim=1).clamp_min(1.0)
        c_pool = (c_emb * c_mask).sum(dim=1) / c_mask.sum(dim=1).clamp_min(1.0)
        parts = [it]
        if self.cfg.use_relations: parts.extend([pb, a_pool, c_pool])
        if self.cfg.use_side:
            parts.extend([dense, self.text_proj(text_emb)])
        x = torch.cat(parts, dim=1)
        return F.normalize(self.item_mlp(x), dim=1)

    def encode_user(self, user_idx, hist_item_vec):
        u = self.user_emb(user_idx)
        x = torch.cat([u, hist_item_vec], dim=1)
        v = self.user_mlp(x)
        if not self.cfg.use_interaction:
            v = self.global_user.unsqueeze(0).expand(user_idx.size(0), -1)
        return F.normalize(v, dim=1)


def build_book_side_indices(book_ids, b2a, b2c, b2p, a2idx, c2idx, p2idx, max_authors=5, max_cats=5):
    B = len(book_ids)
    auth = np.zeros((B, max_authors), dtype=np.int64)
    cats = np.zeros((B, max_cats), dtype=np.int64)
    pubs = np.zeros((B,), dtype=np.int64)
    for j, bid in enumerate(book_ids):
        a_list = b2a.get(bid, [])[:max_authors]
        c_list = b2c.get(bid, [])[:max_cats]
        p = b2p.get(bid, None)
        for k, aid in enumerate(a_list): auth[j, k] = a2idx.get(aid, 0)
        for k, cid in enumerate(c_list): cats[j, k] = c2idx.get(cid, 0)
        pubs[j] = p2idx.get(p, 0) if p is not None else 0
    return auth, cats, pubs

def build_user_histories(train_df, max_k):
    hist = {}
    for u, g in train_df.groupby("u_idx"): hist[u] = g["i_idx"].tolist()[-max_k:]
    return hist

def mrr_at_k(ranked_items, pos_item, k):
    topk = ranked_items[:k]
    return 1.0 / (topk.index(pos_item) + 1) if pos_item in topk else 0.0

def ndcg_at_k(ranked_items, pos_item, k):
    topk = ranked_items[:k]
    return 1.0 / math.log2(topk.index(pos_item) + 2) if pos_item in topk else 0.0

def evaluate_split(model, df, user_hist, all_item_vec, cfg, num_items):
    model.eval()
    hits, ndcgs, mrrs = {k: [] for k in cfg.eval_ks}, {k: [] for k in cfg.ndcg_ks}, []
    with torch.no_grad():
        for row in tqdm(df.itertuples(index=False), desc="Eval", leave=False):
            u, pos = int(row.u_idx), int(row.i_idx)
            h_items = user_hist.get(u, [])
            if not h_items:
                h_vec = torch.zeros((1, cfg.out_dim), device=cfg.device)
            else:
                h_vec = all_item_vec[torch.tensor(h_items, device=cfg.device, dtype=torch.long)].mean(dim=0, keepdim=True)
            uvec = model.encode_user(torch.tensor([u], device=cfg.device), h_vec)
            negs = set()
            while len(negs) < cfg.num_neg_eval:
                ni = random.randrange(num_items)
                if ni != pos: negs.add(ni)
            cand = [pos] + list(negs)
            cvec = all_item_vec[torch.tensor(cand, device=cfg.device)]
            scores = (uvec @ cvec.t()).squeeze(0).cpu().numpy()
            order = np.argsort(-scores)
            ranked = [cand[i] for i in order]
            for k in cfg.eval_ks: hits[k].append(1.0 if pos in ranked[:k] else 0.0)
            for k in cfg.ndcg_ks: ndcgs[k].append(ndcg_at_k(ranked, pos, k))
            mrrs.append(mrr_at_k(ranked, pos, cfg.mrr_k))
    return {**{f"Hit@{k}": np.mean(hits[k]) for k in cfg.eval_ks}, **{f"NDCG@{k}": np.mean(ndcgs[k]) for k in cfg.ndcg_ks}, f"MRR@{cfg.mrr_k}": np.mean(mrrs)}

# ----------------------------
# Main
# ----------------------------
def main(paths: Paths, cfg: TrainConfig):
    set_seed(cfg.seed)
    inter = build_interactions(paths)
    books, authors, cats, pubs, _ = build_book_tables(paths)
    b2a, b2c, b2p = build_multi_maps(paths)

    min_user_inter = 3
    user_counts = inter["user_id"].value_counts()
    inter = inter[inter["user_id"].isin(user_counts[user_counts >= min_user_inter].index)].copy()

    keep_books = set(inter["book_id"].unique())
    books = books[books["book_id"].isin(keep_books)].copy()
    book_ids, user_ids = books["book_id"].unique().tolist(), inter["user_id"].unique().tolist()

    user2idx, item2idx = {u:i for i,u in enumerate(user_ids)}, {b:i for i,b in enumerate(book_ids)}
    author2idx = {aid:i+1 for i,aid in enumerate(sorted({a for b in book_ids for a in b2a.get(b, [])}))}
    cat2idx = {cid:i+1 for i,cid in enumerate(sorted({c for b in book_ids for c in b2c.get(b, [])}))}
    pub2idx = {pid:i+1 for i,pid in enumerate(sorted({b2p.get(b) for b in book_ids if b2p.get(b)}))}

    numeric_cols = [c for c in ["price","offer_price","pages","rating_value","rating_count","review_count","wished_customer_count","stock_quantity"] if c in books.columns]
    if not numeric_cols: books["dummy_num"] = 0.0; numeric_cols = ["dummy_num"]
    books_num = books[["book_id"] + numeric_cols].copy()
    books_num, _ = normalize_numeric(books_num, numeric_cols)
    dense_mat = books_num.set_index("book_id").loc[book_ids].to_numpy(dtype=np.float32)

    if cfg.use_side:
        authors_by_id = {row["author_id"]: row for row in authors.to_dict("records") if "author_id" in row}
        cats_by_id = {row["category_id"]: row for row in cats.to_dict("records")}
        pubs_by_id = {row["publisher_id"]: row for row in pubs.to_dict("records")}
        book_by_id = books.set_index("book_id").to_dict("index")
        book_texts = [make_book_text(pd.Series(book_by_id[bid]), authors_by_id, cats_by_id, pubs_by_id, b2a.get(bid, []), b2c.get(bid, []), b2p.get(bid)) for bid in tqdm(book_ids, desc="Assemble text")]
        text_cache = os.path.join(cfg.text_cache_dir, f"book_text_{cfg.text_model_name.replace('/','_')}_{cfg.ablation_name}.npy")
        text_embs = compute_or_load_text_embeddings(cfg.text_model_name, text_cache, book_texts).astype(np.float32)
    else:
        text_embs = np.zeros((len(book_ids), cfg.text_emb_dim), dtype=np.float32)

    auth_idx, cat_idx, pub_idx = build_book_side_indices(book_ids, b2a, b2c, b2p, author2idx, cat2idx, pub2idx)

    train_df, temp_df = train_test_split(inter, test_size=0.30, random_state=cfg.seed)
    val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=cfg.seed)
    test_df = test_df[test_df["user_id"].isin(user_counts[user_counts <= 10].index)].copy()

    for df in [train_df, val_df, test_df]:
        df["u_idx"], df["i_idx"] = df["user_id"].map(user2idx), df["book_id"].map(item2idx)

    user_hist = build_user_histories(train_df, cfg.max_history)
    dense_t, text_t, auth_t, cat_t, pub_t = [torch.from_numpy(x).to(cfg.device).float() if x.dtype == np.float32 else torch.from_numpy(x).to(cfg.device) for x in [dense_mat, text_embs, auth_idx, cat_idx, pub_idx]]

    model = TwoTower(len(user_ids), len(book_ids), len(author2idx)+1, len(cat2idx)+1, len(pub2idx)+1, dense_t.shape[1], text_t.shape[1], cfg).to(cfg.device)
    opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    train_loader = torch.utils.data.DataLoader(InteractionDataset(train_df, user2idx, item2idx), batch_size=cfg.batch_size, shuffle=True, drop_last=True)

    best_val, best_state, patience_ctr = -1.0, None, 0
    for epoch in range(1, cfg.epochs+1):
        model.train(); total_loss = 0.0
        for u_idx, pos_i, w in tqdm(train_loader, desc=f"Epoch {epoch}"):
            u_idx, pos_i, w = u_idx.to(cfg.device), pos_i.to(cfg.device), w.to(cfg.device)
            pos_vec = model.encode_item(pos_i, pub_t[pos_i], auth_t[pos_i], cat_t[pos_i], dense_t[pos_i], text_t[pos_i])
            h_vecs = []
            for u in u_idx.cpu().numpy():
                h_items = user_hist.get(u, [])
                if not h_items: h_vecs.append(torch.zeros((1, cfg.out_dim), device=cfg.device))
                else:
                    hi = torch.tensor(h_items, device=cfg.device)
                    h_vecs.append(model.encode_item(hi, pub_t[hi], auth_t[hi], cat_t[hi], dense_t[hi], text_t[hi]).mean(dim=0, keepdim=True))
            user_vec = model.encode_user(u_idx, torch.cat(h_vecs, dim=0))
            loss = (F.cross_entropy(user_vec @ pos_vec.t(), torch.arange(cfg.batch_size, device=cfg.device), reduction="none") * w).mean()
            opt.zero_grad(); loss.backward(); opt.step(); total_loss += loss.item()

        model.eval()
        with torch.no_grad():
            all_i = torch.arange(len(book_ids), device=cfg.device)
            all_item_vec = model.encode_item(all_i, pub_t[all_i], auth_t[all_i], cat_t[all_i], dense_t[all_i], text_t[all_i])
        val_m = evaluate_split(model, val_df, user_hist, all_item_vec, cfg, len(book_ids))
        print(f"Epoch {epoch} loss: {total_loss/len(train_loader):.4f} | Val NDCG@10: {val_m['NDCG@10']:.4f}")
        if val_m['NDCG@10'] > best_val: best_val, best_state, patience_ctr = val_m['NDCG@10'], {k: v.cpu().clone() for k, v in model.state_dict().items()}, 0
        else:
            patience_ctr += 1
            if patience_ctr >= cfg.patience: break

    if best_state: model.load_state_dict(best_state)
    with torch.no_grad(): all_item_vec = model.encode_item(all_i, pub_t[all_i], auth_t[all_i], cat_t[all_i], dense_t[all_i], text_t[all_i])
    test_m = evaluate_split(model, test_df, user_hist, all_item_vec, cfg, len(book_ids))
    print("\n=== TEST RESULTS ==="); [print(f"{k:8}: {v:.4f}") for k, v in test_m.items()]

if __name__ == "__main__":
    base_dir = DATA_FOLDER
    paths = Paths(author_json=os.path.join(base_dir, "author.json"), book_json=os.path.join(base_dir, "book.json"), category_json=os.path.join(base_dir, "category.json"), publisher_json=os.path.join(base_dir, "publisher.json"), review_json=os.path.join(base_dir, "review.json"), book_to_author_json=os.path.join(base_dir, "book_to_author.json"), book_to_category_json=os.path.join(base_dir, "book_to_category.json"), book_to_publisher_json=os.path.join(base_dir, "book_to_publisher.json"), book_to_review_json=os.path.join(base_dir, "book_to_review.json"), user_to_review_json=os.path.join(base_dir, "user_to_review.json"))
    main(paths, TrainConfig())